In [1]:
import numpy as np
import torch
from utils import datasets
import gzip
import csv

from network import ConvNet_test
from Chip import Chip_test, chipseq_dataset
from torch.utils.data import DataLoader
from sklearn import metrics
from utils import dinuc_shuffle, reverse_complement, seq2pad

import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.stats import bernoulli

In [22]:
motif_len = 24

In [2]:
# dataset
path = './data/encode/'
dataset_names = datasets(path)
dataset = dataset_names[1][2]

In [3]:
class ConvNet_seq_logo(nn.Module):
    def __init__(self,nummotif,motiflen,poolType,neuType,mode,dropprob, learning_rate,learning_Momentum,sigmaConv,sigmaNeu,beta1,beta2,beta3, device, reverse_complemet_mode):
        super(ConvNet_seq_logo, self).__init__()
        self.poolType=poolType
        self.neuType=neuType
        self.mode=mode
        self.learning_rate=learning_rate
        self.device = device
        self.reverse_complemet_mode=reverse_complemet_mode
        self.momentum_rate=learning_Momentum
        self.sigmaConv=sigmaConv

        self.wConv=torch.randn(nummotif,4,motiflen).to(device)
        torch.nn.init.normal_(self.wConv,mean=0,std=self.sigmaConv)
        self.wConv.requires_grad=True

        self.wRect=torch.randn(nummotif).to(device)
        torch.nn.init.normal_(self.wRect)
        self.wRect=-self.wRect
        self.wRect.requires_grad=True

        self.dropprob=dropprob
        self.sigmaNeu=sigmaNeu
        self.wHidden=torch.randn(2*nummotif,32).to(device)
        self.wHiddenBias=torch.randn(32).to(device)

        if neuType=='nohidden':
            if poolType=='maxavg':
                self.wNeu=torch.randn(2*nummotif,1).to(device)
            else:
                self.wNeu=torch.randn(nummotif,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)
            torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)

        else:
            if poolType=='maxavg':
                self.wHidden=torch.randn(2*nummotif,32).to(device)
            else:
                
                self.wHidden=torch.randn(nummotif,32).to(device)
            self.wNeu=torch.randn(32,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)
            self.wHiddenBias=torch.randn(32).to(device)
            torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wHidden,mean=0,std=0.3)
            torch.nn.init.normal_(self.wHiddenBias,mean=0,std=0.3)
            
  
            self.wHidden.requires_grad=True
            self.wHiddenBias.requires_grad=True
            #wHiddenBias=tf.Variable(tf.truncated_normal([32,1],mean=0,stddev=sigmaNeu)) #hidden bias for everything

        self.wNeu.requires_grad=True
        self.wNeuBias.requires_grad=True

        self.beta1=beta1
        self.beta2=beta2
        self.beta3=beta3
    
    def divide_two_tensors(self,x):
        l=torch.unbind(x)
        list1=[l[2*i] for i in range(int(x.shape[0]/2))]
        list2=[l[2*i+1] for i in range(int(x.shape[0]/2))]
        x1=torch.stack(list1,0)
        x2=torch.stack(list2,0)
        return x1,x2

    def forward_pass(self,x,mask=None,use_mask=False):
        conv=F.conv1d(x, self.wConv, bias=self.wRect, stride=1, padding=0)
        rect=conv.clamp(min=0)

        return rect
        
    def forward(self, x):
        
        if not  self.reverse_complemet_mode:
            out= self.forward_pass(x)
        else:
            print("not supported error")
             
            # x1,x2=self.divide_two_tensors(x)
            # out1,mask=self.forward_pass(x1)
            # out2,_=self.forward_pass(x2,mask,True)
            # out=torch.max(out1, out2)
        
        return out

In [4]:
# Deep Bind Model

name = dataset_names[0][2]
name = name.split(path)[1].split("_AC")[0]

reverse_mode = False

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

checkpoint = torch.load('./Models/'+name+'.pth')

best_hyperparameters = torch.load('./Hyperparameters/'+name+'.pth')

best_poolType=best_hyperparameters['best_poolType']
best_neuType=best_hyperparameters['best_neuType']
best_learning_steps=best_hyperparameters['best_learning_steps']
best_lr=best_hyperparameters['best_LearningRate']
best_droprate=best_hyperparameters['best_dropprob']
best_momentum=best_hyperparameters['best_LearningMomentum']
best_sigmaConv=best_hyperparameters['best_sigmaConv']
best_sigmaNeu=best_hyperparameters['best_sigmaNeu']
best_beta1=best_hyperparameters['best_beta1']
best_beta2=best_hyperparameters['best_beta2']
best_beta3=best_hyperparameters['best_beta3']

model = ConvNet_seq_logo(16, 24, best_poolType, best_neuType, 'test', best_lr, best_momentum, best_sigmaConv, best_droprate, best_sigmaNeu, best_beta1, best_beta2, best_beta3, device, reverse_mode).to(device)
model.wConv=checkpoint['conv']
model.wRect=checkpoint['rect']
model.wHidden=checkpoint['wHidden']
model.wHiddenBias=checkpoint['wHiddenBias']
model.wNeu=checkpoint['wNeu']
model.wNeuBias=checkpoint['wNeuBias']

In [5]:
test_seq = []
reverse_complete_mode = False

# with gzip.open(dataset, 'rt') as data:
#     next(data)
#     reader = csv.reader(data, delimiter = '\t')
#     if not reverse_mode:
#         for row in reader:
#             test_seq.append([row[2], [1]]) # target = 1
#             test_seq.append([dinuc_shuffle(row[2]), [0]]) # target = 0
#     else:
#         for row in reader:
#                 test_seq.append([row[2],[1]])
#                 test_seq.append([reverse_complement(row[2]),[1]])
#                 test_seq.append([dinuc_shuffle(row[2]),[0]])
#                 test_seq.append([dinuc_shuffle(reverse_complement(row[2])),[0]])

# test_dataset=[]

with gzip.open(dataset, 'rt') as data:
    next(data)
    reader = csv.reader(data,delimiter='\t')
    if not reverse_complete_mode:
        for row in reader:
                test_seq.append([row[2],[int(row[3])]])
    else:
        for row in reader:
                test_seq.append([row[2],[int(row[3])]])
                test_seq.append([reverse_complement(row[2]),[int(row[3])]])


In [14]:
test_data = []

for l in range(len(test_seq)):
    test_data.extend([[seq2pad(test_seq[l][0], motiflen=24), test_seq[l][1]]])

In [18]:
test_dataset=chipseq_dataset(test_data)
batchSize=test_dataset.__len__()
test_loader = DataLoader(dataset=test_dataset,batch_size=batchSize,shuffle=False)


In [20]:
with torch.no_grad():
    model.mode = 'test'
    for i, (data, target) in enumerate(test_loader):
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        

In [35]:
i = 0 # which data : 1 ~ 1000
k = 0 # motif_detector number
output[i][k] # Y_ik 

zero_seq = []

for i in range(len(test_data)):
    for k in range(16):
        if(np.argmax(output[i][k].cpu().numpy()) <= 0):
            zero_seq.extend([[i, k]])

In [36]:
len(zero_seq)

24

In [42]:
net_seq = []

for i in range(len(test_data)):
    for k in range(16):
        if([i,k] not in zero_seq):
            net_seq.extend([[i, k]])

In [43]:
len(net_seq)

15976

In [45]:
argmax_position = []

for i,k in net_seq:
    argmax_position.append(np.argmax(output[i][k].cpu().numpy()))

In [47]:
len(argmax_position)

15976

In [49]:
len(test_seq)

1000